In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Human
## Madissoon_Oesophagus

In [ ]:
madissoon_oesophagus = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Madissoon_Oesophagus/data.h5")
utils.peek(madissoon_oesophagus, "build/oesophagus/Madissoon_Oesophagus")
madissoon_oesophagus.obs.head()

In [ ]:
madissoon_oesophagus.obs.dtypes

In [ ]:
madissoon_oesophagus.uns["scanpy_genes"].shape

In [ ]:
opt_model_kwargs = dict(prob_module_kwargs=dict(lambda_reg=0.01), batch_effect="donor")
madissoon_oesophagus_model = cb.directi.fit_DIRECTi(
    madissoon_oesophagus, madissoon_oesophagus.uns["scanpy_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
madissoon_oesophagus.latent = madissoon_oesophagus_model.inference(madissoon_oesophagus)

In [ ]:
ax = madissoon_oesophagus.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/oesophagus/Madissoon_Oesophagus/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = madissoon_oesophagus.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/oesophagus/Madissoon_Oesophagus/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = madissoon_oesophagus.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/oesophagus/Madissoon_Oesophagus/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = madissoon_oesophagus.visualize_latent("sample", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/oesophagus/Madissoon_Oesophagus/sample.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
madissoon_oesophagus.write_dataset("build/oesophagus/Madissoon_Oesophagus/Madissoon_Oesophagus.h5")

In [ ]:
%%capture capio
madissoon_oesophagus_models = [madissoon_oesophagus_model]
opt_model_kwargs = dict(prob_module_kwargs=dict(lambda_reg=0.01))
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    madissoon_oesophagus_models.append(cb.directi.fit_DIRECTi(
        madissoon_oesophagus, madissoon_oesophagus.uns["scanpy_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
madissoon_oesophagus_blast = cb.blast.BLAST(
    madissoon_oesophagus_models, madissoon_oesophagus, 
)
madissoon_oesophagus_blast.save("build/oesophagus/Madissoon_Oesophagus")

In [ ]:
with open("build/oesophagus/Madissoon_Oesophagus/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/oesophagus/Madissoon_Oesophagus/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(madissoon_oesophagus_blast, "build/oesophagus/Madissoon_Oesophagus")

In [ ]:
%%writefile "build/oesophagus/Madissoon_Oesophagus/predictable.txt"
cell_ontology_class
cell_type1